<a href="https://colab.research.google.com/github/annatachibana/quant-macro/blob/main/q_macro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: １．国の選択とGDPデータの取得: 国を1か国選び、FRED（Federal Reserve Economic Data）から四半期の実質GDPデータを取得。
# ２．対数変換: GDPデータに自然対数変換を適用。
# ３．HPフィルターの適用とλの検討: 対数処理したGDPデータに対して、ホドリック＝プレスコット（HP）フィルターを適用し、トレンド成分と循環成分を抽出。
# 複数の異なるスムージングパラメーター(λ=10, 100, 1600)の値を試し、その結果を比較検討してください。λの値を変えることで抽出されるトレンドと循環成分がどのように変化するのかを考察します。
# ４．可視化:
# グラフ1：元のデータとトレンド成分の比較:
# 対数変換後の元のデータ系列と、異なるλ値（λ = 10, 100, 1600）それぞれに対応するトレンド成分を一つのグラフに重ねてプロットしてください。
# 各系列が明確に区別できるように、凡例、軸ラベル、タイトルを適切に付与してください。
# グラフ2：循環成分の比較:
# 異なるλ値（λ = 10, 100, 1600）それぞれに対応する循環成分を一つのグラフに重ねてプロットしてください。
# 各系列が明確に区別できるように、凡例、軸ラベル、タイトルを適切に付与してください。
# これらのグラフを用いて、λの値がトレンドの滑らかさ、循環成分の振幅と頻度に与える影響を視覚的に示してください。

!pip install fredapi statsmodels matplotlib

import pandas as pd
from fredapi import Fred
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Replace with your FRED API key. You can get one for free from https://fred.stlouisfed.org/docs/api/api_key.html
# If you don't have an API key, you can still run the code but you might face rate limits.
fred_api_key = 'YOUR_FRED_API_KEY'
fred = Fred(api_key=fred_api_key)

# 1. 国の選択とGDPデータの取得: 国を1か国選び、FREDから四半期の実質GDPデータを取得。
# Choose a country and the corresponding FRED series ID for real GDP (quarterly).
# Example: United States Real Gross Domestic Product, 3 Decimal Places (GDP)
country_name = 'United States'
gdp_series_id = 'GDPC1'

# Fetch GDP data
gdp_data = fred.get_series(gdp_series_id)
gdp_data = pd.DataFrame(gdp_data, columns=['GDP'])

# Filter out potential NaNs at the beginning
gdp_data = gdp_data.dropna()

# 2. 対数変換: GDPデータに自然対数変換を適用。
log_gdp = np.log(gdp_data['GDP'])

# 3. HPフィルターの適用とλの検討:
lambdas = [10, 100, 1600]
hp_results = {}

for lambda_val in lambdas:
    trend, cycle = sm.tsa.filters.hpfilter(log_gdp, lamb=lambda_val)
    hp_results[lambda_val] = {'trend': trend, 'cycle': cycle}

# 4. 可視化:

# グラフ1：元のデータとトレンド成分の比較:
plt.figure(figsize=(12, 6))
plt.plot(log_gdp.index, log_gdp, label='Log GDP', linewidth=2)
for lambda_val in lambdas:
    plt.plot(log_gdp.index, hp_results[lambda_val]['trend'], label=f'Trend (λ={lambda_val})', linestyle='--')

plt.title('Log GDP vs. HP Filtered Trend Components for different λ')
plt.xlabel('Date')
plt.ylabel('Log GDP')
plt.legend()
plt.grid(True)
plt.show()

# グラフ2：循環成分の比較:
plt.figure(figsize=(12, 6))
for lambda_val in lambdas:
    plt.plot(log_gdp.index, hp_results[lambda_val]['cycle'], label=f'Cycle (λ={lambda_val})')

plt.title('HP Filtered Cycle Components for different λ')
plt.xlabel('Date')
plt.ylabel('Cycle Component')
plt.legend()
plt.grid(True)
plt.show()